In [2]:
import glob
from tqdm.auto import tqdm

from pymatgen.io.vasp import Outcar, Poscar
import numpy as np

from ase import Atoms, units
from ase.io import write, read
from ase.calculators.singlepoint import SinglePointCalculator

In [9]:

def outcar2traj(symbols: list[str] | None, outcar: Outcar, log: bool = False) -> list[Atoms]:

    def str2floats(s: str):
        return [float(x) for x in s.split()]
    
    # energy

    row_pattern = r'energy without entropy\s*=\s*(-?\d+\.\d+)\s+energy\(sigma-\>0\)\s*=\s*(-?\d+\.\d+)'

    data = outcar.read_table_pattern(
        header_pattern=r'\s*',
        row_pattern=row_pattern,
        footer_pattern=r'\s*',
        postprocess=str2floats,
        last_one_only=False,
        first_one_only=False,
    )

    data = np.array(data).squeeze()

    outcar.data['energy'] = data[:, -1]

    # positions and forces

    header_pattern = r'^POSITION\s*TOTAL-FORCE \(eV\/Angst\)\s*\n\s*-+\s*'
    row_pattern = r'\s*(-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+)\s*'

    data = outcar.read_table_pattern(
        header_pattern=r'\s*-+\s*',
        row_pattern=row_pattern,
        footer_pattern=r'\s*-+\s*',
        postprocess=str2floats,
        last_one_only=False,
        first_one_only=False,
    )
    data = np.array(data).squeeze()

    outcar.data['positions'] = data[:, :, :3]
    outcar.data['forces'] = data[:, :, 3:]

    # magmoms

    # Magnetization
    header_pattern = r'magnetization \(x\)\s*\n\s*\n\s*# of ion\s+s\s+p\s+d\s+f\s+tot\s*\n\s*-+\s*'
    row_pattern = r'\s*(\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+\s+-?\d+\.\d+)\s*'

    data = outcar.read_table_pattern(
        header_pattern=header_pattern,
        row_pattern=row_pattern,
        footer_pattern=r'\s*-+\s*',
        postprocess=str2floats,
        last_one_only=False,
        first_one_only=False,
    )
    
    data = np.array(data).squeeze()
    outcar.data['magmom_x'] = data[:, :, -1]  # Exclude the ion number column


    # lattice

    header_pattern = r'\s*direct lattice vectors\s*reciprocal lattice vectors'
    row_pattern = r'\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*(-?\d+\.\d+)\s*'

    data = outcar.read_table_pattern(
        header_pattern=header_pattern, # r'\s*-+\s*',
        row_pattern=row_pattern,
        footer_pattern=r'\s*',
        postprocess=str2floats,
        last_one_only=False,
        first_one_only=False,
    )
    data = np.array(data).squeeze()

    outcar.data['lattice'] = data[:, :, :3]

    pattern_stress = r'in kB\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)\s+(-?\d+\.\d+)'

    outcar.read_pattern({
        "stress": pattern_stress,
    })

    outcar.data['stress'] = np.array(outcar.data['stress'], dtype=np.float32) * -1e-1 * units.GPa

    # nsteps = min(outcar.data['positions'].shape[0], outcar.data['energy'].shape[0], outcar.data['forces'].shape[0], outcar.data['stress'].shape[0], outcar.data['lattice'].shape[0])

    outcar.data = {
        key: value for key, value in outcar.data.items() if key in [
            'positions', 'energy', 'forces', 'magmom_x', 'stress', 'lattice'
        ]
    }

    nsteps = min(map(len, outcar.data.values()))

    for key in outcar.data:
        outcar.data[key] = outcar.data[key][:nsteps]

    if log:
        print('nsteps', nsteps)
        for key, value in outcar.data.items():
            print(key, value.shape)

    traj = []

    
    for i in tqdm(range(nsteps)):

        atoms = Atoms(
            symbols=symbols,
            positions=outcar.data['positions'][i], 
            cell=outcar.data['lattice'][i],
            pbc=True,
        )

        magmoms = outcar.data.get('magmoms', None) or outcar.data.get('magmom_x', None)

        atoms.calc = SinglePointCalculator(
            atoms, 
            energy=outcar.data['energy'][i], 
            forces=outcar.data['forces'][i], 
            magmoms=magmoms,
            stress=outcar.data['stress'][i],
        )

        traj.append(atoms)

    return traj

In [11]:
outcar = Outcar('OUTCAR.gz')

traj = outcar2traj(None, outcar, log=True)

nsteps 9
energy (9,)
positions (9, 72, 3)
forces (9, 72, 3)
magmom_x (9, 72)
lattice (9, 3, 3)
stress (9, 6)


  0%|          | 0/9 [00:00<?, ?it/s]